## Fetch Weather Data: API Exploration
### *This notebook fetches weather data from two sources: NOAA and Open Weather*

#### NOAA API
The documentation for the web API for NOAA climate data can be found [here](https://www.ncdc.noaa.gov/cdo-web/webservices/v2).

In [4]:
import requests
import json
import pandas as pd
from pyproj import Proj
from shapely.geometry import shape
import pickle
#headers = {"token": "xVEIkLnfHyheHhvoheZSxesUerlyrxGN"}
headers = {"token": "bgBKBpxaFICMeDMgPFYAMGyaDrBJMPwB"}

#### All Available Datasets

In [5]:
url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/datasets"
response = requests.get(url=url, headers=headers)
noaa_all_datasets_json = response.json()
print(noaa_all_datasets_json["metadata"])
noaa_all_datasets_df = pd.DataFrame(noaa_all_datasets_json['results'])
noaa_all_datasets_df

{'resultset': {'offset': 1, 'count': 11, 'limit': 25}}


,datacoverage,id,maxdate,mindate,name,uid
0,1.00,GHCND,2018-10-17,1763-01-01,Daily Summaries,gov.noaa.ncdc:C00861
1,1.00,GSOM,2018-09-01,1763-01-01,Global Summary of the Month,gov.noaa.ncdc:C00946
2,1.00,GSOY,2018-01-01,1763-01-01,Global Summary of the Year,gov.noaa.ncdc:C00947
3,0.95,NEXRAD2,2018-10-18,1991-06-05,Weather Radar (Level II),gov.noaa.ncdc:C00345
4,0.95,NEXRAD3,2018-10-15,1994-05-20,Weather Radar (Level III),gov.noaa.ncdc:C00708
5,1.00,NORMAL_ANN,2010-01-01,2010-01-01,Normals Annual/Seasonal,gov.noaa.ncdc:C00821
6,1.00,NORMAL_DLY,2010-12-31,2010-01-01,Normals Daily,gov.noaa.ncdc:C00823
7,1.00,NORMAL_HLY,2010-12-31,2010-01-01,Normals Hourly,gov.noaa.ncdc:C00824
8,1.00,NORMAL_MLY,2010-12-01,2010-01-01,Normals Monthly,gov.noaa.ncdc:C00822
9,0.25,PRECIP_15,2014-01-01,1970-05-12,Precipitation 15 Minute,gov.noaa.ncdc:C00505


#### Daily Summaries Dataset

* For our purposes, we will be working with daily summaries data.
* Fetch all information about the GHCND, Daily Summaries dataset specifically.

In [6]:
url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/datasets/GHCND"
response = requests.get(url=url, headers=headers)
noaa_daily_summaries_json = response.json()

In [7]:
print(noaa_daily_summaries_json)

{'mindate': '1763-01-01', 'maxdate': '2018-10-17', 'name': 'Daily Summaries', 'datacoverage': 1, 'id': 'GHCND'}


#### Datatype Filter

In [8]:
url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/datasets?datatypeid=TOBS"
response = requests.get(url=url, headers=headers)
noaa_tobs_json = response.json()

In [9]:
noaa_tobs_json

{'metadata': {'resultset': {'count': 1, 'limit': 25, 'offset': 1}},
 'results': [{'datacoverage': 1,
   'id': 'GHCND',
   'maxdate': '2018-10-17',
   'mindate': '1763-01-01',
   'name': 'Daily Summaries',
   'uid': 'gov.noaa.ncdc:C00861'}]}

#### Set of Stations

In [10]:
url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/datasets?stationid=COOP:310090&stationid=COOP:310184&stationid=COOP:310212"
response = requests.get(url=url, headers=headers)
noaa_stations_json = response.json()
print(noaa_stations_json)

{}


Note: The above example is not working as there is no data available at the given list of stations at this moment.

#### Data Categories

In [11]:
url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/datacategories?limit=100"
response = requests.get(url=url, headers=headers)
noaa_data_categories_json = response.json()
print(noaa_data_categories_json["metadata"])
noaa_data_categories_df = pd.DataFrame(noaa_data_categories_json['results'])
print(noaa_data_categories_df.shape)
noaa_data_categories_df

{'resultset': {'offset': 1, 'count': 42, 'limit': 100}}
(42, 2)


,id,name
0,ANNAGR,Annual Agricultural
1,ANNDD,Annual Degree Days
2,ANNPRCP,Annual Precipitation
3,ANNTEMP,Annual Temperature
4,AUAGR,Autumn Agricultural
5,AUDD,Autumn Degree Days
6,AUPRCP,Autumn Precipitation
7,AUTEMP,Autumn Temperature
8,COMP,Computed
9,COMPAGR,Computed Agricultural


#### WIND Datacategory

* We will be taking the WIND data category for our analysis in the preliminary phase.
* Fetch all information about the Wind dataset specifically

In [12]:
url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/datacategories/WIND"
response = requests.get(url=url, headers=headers)
noaa_wind_json = response.json()

In [13]:
print(noaa_wind_json)

{'status': '429', 'message': 'This token has reached its temporary request limit of 5 per second.'}


#### Datatypes

In [14]:
url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/datatypes"
response = requests.get(url=url, headers=headers)
noaa_data_types_json = response.json()
print(noaa_data_types_json["metadata"])
noaa_data_types_df = pd.DataFrame(noaa_data_types_json['results'])
print(noaa_data_types_df.shape)
noaa_data_types_df

{'resultset': {'offset': 1, 'count': 1527, 'limit': 25}}
(25, 5)


,datacoverage,id,maxdate,mindate,name
0,1.00,ACMC,1996-05-28,1994-03-19,Average cloudiness midnight to midnight from 3...
1,1.00,ACMH,2005-12-31,1965-01-01,Average cloudiness midnight to midnight from m...
2,1.00,ACSC,1996-05-28,1994-02-01,Average cloudiness sunrise to sunset from 30-s...
3,1.00,ACSH,2005-12-31,1965-01-01,Average cloudiness sunrise to sunset from manu...
4,0.95,ALL,2018-10-18,1991-06-05,Base Data
5,1.00,ANN-CLDD-BASE45,2010-01-01,2010-01-01,Long-term averages of annual cooling degree da...
6,1.00,ANN-CLDD-BASE50,2010-01-01,2010-01-01,Long-term averages of annual cooling degree da...
7,1.00,ANN-CLDD-BASE55,2010-01-01,2010-01-01,Long-term averages of annual cooling degree da...
8,1.00,ANN-CLDD-BASE57,2010-01-01,2010-01-01,Long-term averages of annual cooling degree da...
9,1.00,ANN-CLDD-BASE60,2010-01-01,2010-01-01,Long-term averages of annual cooling degree da...


#### Datatype in Wind Category

In [15]:
url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/datatypes?datacategoryid=WIND&limit=56"
response = requests.get(url=url, headers=headers)
noaa_wind_data_types_json = response.json()
print(noaa_wind_data_types_json["metadata"])
noaa_wind_data_types_df = pd.DataFrame(noaa_wind_data_types_json['results'])
print(noaa_wind_data_types_df.shape)
noaa_wind_data_types_df

{'resultset': {'offset': 1, 'count': 27, 'limit': 56}}
(27, 5)


,datacoverage,id,maxdate,mindate,name
0,1,AWND,2018-10-16,1982-01-01,Average wind speed
1,1,DAWM,2010-06-21,1935-09-23,Number of days included in the multiday wind m...
2,1,FMTM,2013-03-31,1982-01-01,Time of fastest mile or fastest 1-minute wind
3,1,HLY-WIND-1STDIR,2010-12-31,2010-01-01,Prevailing wind direction (1-8)
4,1,HLY-WIND-1STPCT,2010-12-31,2010-01-01,Prevailing wind percentage
5,1,HLY-WIND-2NDDIR,2010-12-31,2010-01-01,Secondary wind direction (1-8)
6,1,HLY-WIND-2NDPCT,2010-12-31,2010-01-01,Secondary wind percentage
7,1,HLY-WIND-AVGSPD,2010-12-31,2010-01-01,Average wind speed
8,1,HLY-WIND-PCTCLM,2010-12-31,2010-01-01,Percentage calm
9,1,HLY-WIND-VCTDIR,2010-12-31,2010-01-01,Mean wind vector direction


#### Locations

In [16]:
url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/locationcategories"
response = requests.get(url=url, headers=headers)
noaa_location_categories_json = response.json()
print(noaa_location_categories_json["metadata"])
noaa_location_categories_df = pd.DataFrame(noaa_location_categories_json['results'])
print(noaa_location_categories_df.shape)
noaa_location_categories_df

{'resultset': {'offset': 1, 'count': 12, 'limit': 25}}
(12, 2)


,id,name
0,CITY,City
1,CLIM_DIV,Climate Division
2,CLIM_REG,Climate Region
3,CNTRY,Country
4,CNTY,County
5,HYD_ACC,Hydrologic Accounting Unit
6,HYD_CAT,Hydrologic Cataloging Unit
7,HYD_REG,Hydrologic Region
8,HYD_SUB,Hydrologic Subregion
9,ST,State


#### Location Category: County Level Information

In [17]:
url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/locationcategories/CNTY"
response = requests.get(url=url, headers=headers)
noaa_cnty_json = response.json()

In [18]:
print(noaa_cnty_json)

{'name': 'County', 'id': 'CNTY'}


#### Available Locations for Daily Summaries Data

In [19]:
url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/locations?datasetid=GHCND"
response = requests.get(url=url, headers=headers)
noaa_daily_summaries_locations_json = response.json()
print(noaa_daily_summaries_locations_json["metadata"])
noaa_daily_summaries_locations_df = pd.DataFrame(noaa_daily_summaries_locations_json['results'])
print(noaa_daily_summaries_locations_df.shape)
noaa_daily_summaries_locations_df

{'resultset': {'offset': 1, 'count': 28356, 'limit': 25}}
(25, 5)


,datacoverage,id,maxdate,mindate,name
0,0.9977,CITY:AE000001,2018-10-16,1983-01-02,"Abu Dhabi, AE"
1,0.9992,CITY:AE000002,2018-10-16,1944-03-20,"Ajman, AE"
2,0.9992,CITY:AE000003,2018-10-16,1944-03-20,"Dubai, AE"
3,0.9992,CITY:AE000006,2018-10-16,1944-03-20,"Sharjah, AE"
4,0.5547,CITY:AF000007,2018-10-16,1966-03-02,"Kabul, AF"
5,0.3781,CITY:AF000008,2018-10-16,1973-01-02,"Kandahar, AF"
6,1.0000,CITY:AG000001,2018-10-16,1877-04-07,"Algiers, AG"
7,0.9252,CITY:AG000002,2018-10-16,1909-11-23,"Annaba, AG"
8,0.8655,CITY:AG000003,2018-10-16,1973-04-03,"Batna, AG"
9,1.0000,CITY:AG000004,2018-10-16,1957-01-09,"Bechar, AG"


#### Available Stations

In [20]:
#url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/locations?datasetid=GHCND"
url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/stations?datasetid=GHCND&datacategoryid=WIND&limit=1000&startdate=2000-01-01&extent=44.3506,-122.3201,46.492,-122.1787"
response = requests.get(url=url, headers=headers)
noaa_daily_summaries_locations_json = response.json()
print(noaa_daily_summaries_locations_json["metadata"])
noaa_daily_summaries_locations_df = pd.DataFrame(noaa_daily_summaries_locations_json['results'])
print(noaa_daily_summaries_locations_df.shape)
noaa_daily_summaries_locations_df

{'resultset': {'offset': 1, 'count': 1, 'limit': 1000}}
(1, 9)


,datacoverage,elevation,elevationUnit,id,latitude,longitude,maxdate,mindate,name
0,0.9903,371.9,METERS,GHCND:USC00352292,44.7241,-122.2547,2018-10-10,1954-11-16,"DETROIT DAM, OR US"


#### Getting Stations for General Electric Project

In [21]:
import numpy as np
import time

In [39]:
def get_weather_stations(lat_center, long_center, square_diagonal, top_n = 5):
    flag = True
    count = 0
    while flag:
        count += 1
        base_url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/stations?datasetid=GHCND&datacategoryid=WIND&limit=1000&startdate=2000-01-01&extent="
        epsilon = round(square_diagonal/np.sqrt(2)/2,4)
        a = [long_center+epsilon, lat_center+epsilon]
        b = [long_center+epsilon, lat_center-epsilon]
        c = [long_center-epsilon, lat_center-epsilon]
        d = [long_center-epsilon, lat_center+epsilon]
        w = (long_center+epsilon, lat_center+epsilon)
        x = (long_center+epsilon, lat_center-epsilon)
        y = (long_center-epsilon, lat_center-epsilon)
        z = (long_center-epsilon, lat_center+epsilon)
        url_a = round(lat_center-epsilon,4)
        url_b = round(long_center-epsilon,4)
        url_c = round(lat_center+epsilon,4)
        url_d = round(long_center+epsilon,4)
        url = base_url + str(url_a) + "," + str(url_b) + "," + str(url_c) + "," + str(url_d)
        response = requests.get(url=url, headers=headers)
        all_stations = response.json()
        try:
            all_stations_results = pd.DataFrame(all_stations['results'])
        except:
            all_stations_results = pd.DataFrame()
        try:
            all_stations_meta = all_stations['metadata']
        except:
            all_stations_meta = None
        n_stations = all_stations_results.shape[0]
        if n_stations >= top_n:
            print('Summary of Request: ')
            print(all_stations_meta)
            print('- . - . - . -')
            n_stations = all_stations_results.shape[0]
            {"type": "Polygon", 
             "coordinates": [[
               a, b, c, d
             ]]}
            co = {"type": "Polygon", "coordinates": [
                [w, x, y, z]]}
            lon, lat = zip(*co['coordinates'][0])
            #pa = Proj("+proj=aea +lat_1=37.0 +lat_2=41.0 +lat_0=39.0 +lon_0=-106.55")
            #x, y = pa(lon, lat)
            #cop = {"type": "Polygon", "coordinates": [zip(x, y)]}
            #final_area = shape(cop).area  # 268952044107.43506 square meters
            #print('Square Meters Area Queried: ')
            #print(final_area)
            #print('- . - . - . -')
            flag = False
            #print('Gathered Sufficient Stations')
        else:
            #print(square_diagonal)
            square_diagonal = square_diagonal*1.2
            #print('Increasing Search Area...')
        time.sleep(0.25)
    all_stations_results['square_diagonal'] = square_diagonal
    all_stations_results['query_count'] = count
    #all_stations_results['final_area'] = final_area
    return all_stations_results

In [40]:
#get_weather_stations(lat_center, long_center, square_diagonal = 0.2, top_n = 5)

#### Query Stations for GE Projects

* This block fetches the center latitudes and longitudes of the projects for General Electric.

In [41]:
usgs_data = pd.read_csv('./uswtdbCSV/uswtdb_v1_1_20180710.csv')
usgs_ge_data = usgs_data[usgs_data["t_manu"] == "GE Wind"]
ge_projects_df = pd.pivot_table(usgs_ge_data, values=["xlong", "ylat"], columns="p_name", aggfunc="mean").transpose()
ge_projects_df.reset_index(inplace=True)
ge_projects_df.columns = ['p_name', 'center_long', 'center_lat']
ge_projects_df.head()

,p_name,center_long,center_lat
0,6th Space Warning Squadron,-70.543552,41.753341
1,AFCEE MMR Turbines,-70.546550,41.758590
2,AG Land 1,-93.325691,42.206390
3,AG Land 2,-93.428093,42.146091
4,AG Land 3,-93.431992,42.145592


* All GE Projects: Range of the turbine locations

In [42]:
usgs_data = pd.read_csv('./uswtdbCSV/uswtdb_v1_1_20180710.csv')
usgs_ge_data = usgs_data[usgs_data["t_manu"] == "GE Wind"]
usgs_ge_data = usgs_ge_data[(usgs_ge_data["case_id"] != 3064510) & (usgs_ge_data["case_id"] != 3064731)]
ge_by_project_df = usgs_ge_data.groupby("p_name")[["xlong", "ylat"]].agg(["min", "max", "mean"])
ge_by_project_df.reset_index(inplace=True)
ge_by_project_df.columns = ["p_name", "long_min", "long_max", "long_mean", "lat_min", "lat_max", "lat_mean"]
ge_by_project_df["long_range"] = ge_by_project_df["long_max"] - ge_by_project_df["long_min"]
ge_by_project_df["lat_range"] = ge_by_project_df["lat_max"] - ge_by_project_df["lat_min"]
ge_by_project_df.sort_values(by="long_range", axis=0, ascending=False, inplace=True)
ge_by_project_df.head()

,p_name,long_min,long_max,long_mean,lat_min,lat_max,lat_mean,long_range,lat_range
373,Thunder Ranch,-97.495476,-97.091446,-97.306297,36.527344,36.632317,36.572365,0.404030,0.104973
287,Peetz Table,-103.529190,-103.137192,-103.322228,40.896992,40.998993,40.961955,0.391998,0.102001
164,Horse Hollow,-100.327034,-99.958275,-100.173965,32.180569,32.305607,32.253115,0.368759,0.125038
39,Brady Wind I,-102.953804,-102.605217,-102.756389,46.632294,46.719784,46.656348,0.348587,0.087490
126,Flat Ridge 2,-98.425491,-98.078690,-98.252666,37.323296,37.410694,37.371157,0.346801,0.087398


#### Fetch the weather stations for the GE Project

In [43]:
all_project_stations = pd.DataFrame()

In [44]:
count = 0
for index, row in ge_by_project_df.iterrows():
    try:
        count += 1
        print(row['p_name'], "| Project # ", count)
        lat_center = row['lat_mean']
        long_center = row['long_mean']
        project_stations = get_weather_stations(lat_center, long_center, square_diagonal = 0.2, top_n = 5)
        project_stations['query_long'] = long_center
        project_stations['query_lat'] = lat_center
        #print(project_stations)
        project_stations['dist_from_center'] = np.sqrt((project_stations['query_lat'] - project_stations['latitude'])**2 + (project_stations['query_long'] - project_stations['longitude'])**2)
        project_stations = project_stations.sort_values('dist_from_center').reset_index()
        project_stations = project_stations.loc[0:2]
        project_stations['p_name'] = row['p_name']
        all_project_stations = all_project_stations.append(project_stations)
    except:
        print('JSON Decode Error!')

Thunder Ranch
Summary of Request: 
{'resultset': {'offset': 1, 'count': 6, 'limit': 1000}}
- . - . - . -
Peetz Table
Summary of Request: 
{'resultset': {'offset': 1, 'count': 5, 'limit': 1000}}
- . - . - . -
Horse Hollow
Summary of Request: 
{'resultset': {'offset': 1, 'count': 6, 'limit': 1000}}
- . - . - . -
Brady Wind I
Summary of Request: 
{'resultset': {'offset': 1, 'count': 5, 'limit': 1000}}
- . - . - . -
Flat Ridge 2
Summary of Request: 
{'resultset': {'offset': 1, 'count': 5, 'limit': 1000}}
- . - . - . -
Prairie Breeze
Summary of Request: 
{'resultset': {'offset': 1, 'count': 5, 'limit': 1000}}
- . - . - . -
Panhandle Wind 1
Summary of Request: 
{'resultset': {'offset': 1, 'count': 7, 'limit': 1000}}
- . - . - . -
Rush Springs Wind Energy Center
Summary of Request: 
{'resultset': {'offset': 1, 'count': 5, 'limit': 1000}}
- . - . - . -
Flat Ridge 2 Expansion
Summary of Request: 
{'resultset': {'offset': 1, 'count': 5, 'limit': 1000}}
- . - . - . -
Chisholm View
Summary of Requ

Tuscola Bay Wind
Summary of Request: 
{'resultset': {'offset': 1, 'count': 9, 'limit': 1000}}
- . - . - . -
Bobcat Bluff
Summary of Request: 
{'resultset': {'offset': 1, 'count': 6, 'limit': 1000}}
- . - . - . -
Klondike III (Wasco)
Summary of Request: 
{'resultset': {'offset': 1, 'count': 8, 'limit': 1000}}
- . - . - . -
Mesquite Creek
Summary of Request: 
{'resultset': {'offset': 1, 'count': 5, 'limit': 1000}}
- . - . - . -
Prairie Breeze II
Summary of Request: 
{'resultset': {'offset': 1, 'count': 8, 'limit': 1000}}
- . - . - . -
Camp Springs II (4Q07)
Summary of Request: 
{'resultset': {'offset': 1, 'count': 5, 'limit': 1000}}
- . - . - . -
Breckinridge
Summary of Request: 
{'resultset': {'offset': 1, 'count': 5, 'limit': 1000}}
- . - . - . -
Blue Summit
Summary of Request: 
{'resultset': {'offset': 1, 'count': 5, 'limit': 1000}}
- . - . - . -
Pioneer Trail
Summary of Request: 
{'resultset': {'offset': 1, 'count': 5, 'limit': 1000}}
- . - . - . -
Horse Creek Wind
Summary of Request

Kit Carson Project
Summary of Request: 
{'resultset': {'offset': 1, 'count': 7, 'limit': 1000}}
- . - . - . -
Big Horn
Summary of Request: 
{'resultset': {'offset': 1, 'count': 6, 'limit': 1000}}
- . - . - . -
Desert Sky
Summary of Request: 
{'resultset': {'offset': 1, 'count': 6, 'limit': 1000}}
- . - . - . -
Notrees
Summary of Request: 
{'resultset': {'offset': 1, 'count': 6, 'limit': 1000}}
- . - . - . -
Sendero
Summary of Request: 
{'resultset': {'offset': 1, 'count': 6, 'limit': 1000}}
- . - . - . -
Spearville 3
Summary of Request: 
{'resultset': {'offset': 1, 'count': 6, 'limit': 1000}}
- . - . - . -
Stephens Ranch 1
Summary of Request: 
{'resultset': {'offset': 1, 'count': 5, 'limit': 1000}}
- . - . - . -
Shepherds Flat (2011)
Summary of Request: 
{'resultset': {'offset': 1, 'count': 5, 'limit': 1000}}
- . - . - . -
Campbell County
Summary of Request: 
{'resultset': {'offset': 1, 'count': 8, 'limit': 1000}}
- . - . - . -
Elm Creek
Summary of Request: 
{'resultset': {'offset': 1,

Laredo Ridge
Summary of Request: 
{'resultset': {'offset': 1, 'count': 8, 'limit': 1000}}
- . - . - . -
Wolverine Creek
Summary of Request: 
{'resultset': {'offset': 1, 'count': 6, 'limit': 1000}}
- . - . - . -
Campbell Hill
Summary of Request: 
{'resultset': {'offset': 1, 'count': 6, 'limit': 1000}}
- . - . - . -
Colorado Highlands (2012)
Summary of Request: 
{'resultset': {'offset': 1, 'count': 7, 'limit': 1000}}
- . - . - . -
Pioneer Wind Park
Summary of Request: 
{'resultset': {'offset': 1, 'count': 5, 'limit': 1000}}
- . - . - . -
Grand Ridge
Summary of Request: 
{'resultset': {'offset': 1, 'count': 5, 'limit': 1000}}
- . - . - . -
Wessington Springs
Summary of Request: 
{'resultset': {'offset': 1, 'count': 5, 'limit': 1000}}
- . - . - . -
Red Mesa
Summary of Request: 
{'resultset': {'offset': 1, 'count': 7, 'limit': 1000}}
- . - . - . -
White Tail
Summary of Request: 
{'resultset': {'offset': 1, 'count': 7, 'limit': 1000}}
- . - . - . -
Laurel Mountain
Summary of Request: 
{'resu

Salmon Falls
Summary of Request: 
{'resultset': {'offset': 1, 'count': 5, 'limit': 1000}}
- . - . - . -
Durbin Creek
Summary of Request: 
{'resultset': {'offset': 1, 'count': 5, 'limit': 1000}}
- . - . - . -
Benson Creek
Summary of Request: 
{'resultset': {'offset': 1, 'count': 5, 'limit': 1000}}
- . - . - . -
Junction Hilltop Wind
Summary of Request: 
{'resultset': {'offset': 1, 'count': 7, 'limit': 1000}}
- . - . - . -
Iowa Lakes Lakota Wind
Summary of Request: 
{'resultset': {'offset': 1, 'count': 7, 'limit': 1000}}
- . - . - . -
Creston Ridge Expansion
Summary of Request: 
{'resultset': {'offset': 1, 'count': 6, 'limit': 1000}}
- . - . - . -
Jett Creek
Summary of Request: 
{'resultset': {'offset': 1, 'count': 5, 'limit': 1000}}
- . - . - . -
Sunray I
Summary of Request: 
{'resultset': {'offset': 1, 'count': 6, 'limit': 1000}}
- . - . - . -
Kaheawa (Maalaea)
Summary of Request: 
{'resultset': {'offset': 1, 'count': 5, 'limit': 1000}}
- . - . - . -
Iowa Lakes Superior Wind
Summary of

Wolverine
Summary of Request: 
{'resultset': {'offset': 1, 'count': 6, 'limit': 1000}}
- . - . - . -
AG Land 2
Summary of Request: 
{'resultset': {'offset': 1, 'count': 8, 'limit': 1000}}
- . - . - . -
AG Land 1
Summary of Request: 
{'resultset': {'offset': 1, 'count': 6, 'limit': 1000}}
- . - . - . -
Walmart Red Bluff
Summary of Request: 
{'resultset': {'offset': 1, 'count': 7, 'limit': 1000}}
- . - . - . -
Central Community College-Hastings
Summary of Request: 
{'resultset': {'offset': 1, 'count': 5, 'limit': 1000}}
- . - . - . -
Meadow Ridge
Summary of Request: 
{'resultset': {'offset': 1, 'count': 6, 'limit': 1000}}
- . - . - . -
Ipswich
Summary of Request: 
{'resultset': {'offset': 1, 'count': 6, 'limit': 1000}}
- . - . - . -
NREL - GE
Summary of Request: 
{'resultset': {'offset': 1, 'count': 5, 'limit': 1000}}
- . - . - . -
Mars Hill
Summary of Request: 
{'resultset': {'offset': 1, 'count': 6, 'limit': 1000}}
- . - . - . -
Luther College
Summary of Request: 
{'resultset': {'offse

Saving dataframe to csv and pickle files.

In [58]:
all_project_stations.reset_index(inplace=True)
all_project_stations.drop(columns=["index", "level_0"], inplace=True)
all_project_stations["id"] = all_project_stations["id"].str.replace(":", "_", regex=False)
all_project_stations["p_name"] = all_project_stations["p_name"].str.replace("/","&", regex=False)
all_project_stations

,datacoverage,elevation,elevationUnit,id,latitude,longitude,maxdate,mindate,name,square_diagonal,query_count,query_long,query_lat,dist_from_center,p_name
0,0.9995,304.8,METERS,GHCND_USW00013969,36.73667,-97.10194,2018-10-16,1948-01-01,"PONCA CITY MUNICIPAL AIRPORT, OK US",2.139864,14,-97.306297,36.572365,0.262218,Thunder Ranch
1,0.9872,299.9,METERS,GHCND_USW00003965,36.16240,-97.08940,2018-10-16,1996-11-01,"STILLWATER REGIONAL AIRPORT, OK US",2.139864,14,-97.306297,36.572365,0.463805,Thunder Ranch
2,0.9675,272.8,METERS,GHCND_USC00348501,36.11750,-97.09500,2018-10-17,1893-01-01,"STILLWATER 2 W, OK US",2.139864,14,-97.306297,36.572365,0.501546,Thunder Ranch
3,0.8607,1309.4,METERS,GHCND_USW00024030,41.09930,-102.98630,2018-10-16,1948-01-01,"SIDNEY MUNICIPAL AIRPORT, NE US",1.783220,13,-103.322228,40.961955,0.362920,Peetz Table
4,0.9565,1316.7,METERS,GHCND_USC00257830,41.22940,-103.02140,2018-09-30,1908-12-01,"SIDNEY 6 NNW, NE US",1.783220,13,-103.322228,40.961955,0.402522,Peetz Table
5,0.9988,1130.5,METERS,GHCND_USC00051996,40.86000,-102.80300,2018-10-17,1996-03-01,"CROOK, CO US",1.783220,13,-103.322228,40.961955,0.529143,Peetz Table
6,0.9804,545.6,METERS,GHCND_USW00013962,32.41050,-99.68220,2018-10-16,1946-08-01,"ABILENE REGIONAL AIRPORT, TX US",2.567837,15,-100.173965,32.253115,0.516336,Horse Hollow
7,0.9411,594.1,METERS,GHCND_USC00414278,31.84555,-99.56068,2018-10-17,1953-07-01,"HORDS CREEK DAM, TX US",2.567837,15,-100.173965,32.253115,0.736361,Horse Hollow
8,0.8222,598.6,METERS,GHCND_USC00416499,31.45389,-100.49333,2008-01-31,1975-12-01,"O C FISHER DAM, TX US",2.567837,15,-100.173965,32.253115,0.860671,Horse Hollow
9,0.8793,786.4,METERS,GHCND_USW00024012,46.79940,-102.79720,2018-10-16,1938-11-14,"DICKINSON THEODORE ROOSEVELT REGIONAL AIRPORT,...",3.081404,16,-102.756389,46.656348,0.148759,Brady Wind I


In [59]:
all_project_stations.to_csv('./data/all_project_stations.csv', index=False)
pickle_out = open("./data/all_project_stations.pickle", "wb")
pickle.dump(all_project_stations, pickle_out)
pickle_out.close()